In [2]:
!pip install pyrosm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 6.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.6 MB 47.2 MB/s 
     |████████████████████████████████| 2.1 MB 47.4 MB/s 
  Using cached pyrobuf-0.9.3-cp37-cp37m-linux_x86_64.whl
  Using cached cykhash-2.0.0-cp37-cp37m-linux_x86_64.whl
     |████████████████████████████████| 1.0 MB 58.5 MB/s 
     |████████████████████████████████| 6.3 MB 34.7 MB/s 
     |████████████████████████████████| 16.7 MB 47.4 MB/s 
  Created wheel for pyrosm: filename=pyrosm-0.6.1-cp37-cp37m-linux_x86_64.whl size=5431194 sha256=4aad89a3fc3ac0e8051b93075d91892af91042677f068db6ed1e655a60f47725
  Stored in directory: /root/.cache/pip/wheels/e8/9e/d8/e424ffc5eaa71ccb122bb79d92ef75802c49b9b0a9366d6da4
Successfully built pyrosm


In [4]:
!pip install rdflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 500 kB 7.8 MB/s 
     |████████████████████████████████| 41 kB 648 kB/s 


In [5]:
# Libraries
import os
from pyrosm import OSM
from pathlib import Path
from rdflib import Graph, Literal, RDF, URIRef, Namespace, FOAF
from rdflib.namespace import XSD

In [60]:
# Parameters and saving folder
path = str(Path(os.path.abspath(os.getcwd())).absolute())
dataset = path + '\\data\\padova-mestre-veneto.osm.pbf'
savepath = path + 'C:\\Users\\Aliia\\PycharmProjects\\boring-wozniak\\data\\padova-mestre-veneto.osm.pbf'

In [61]:
# Loading dataset
from pyrosm import OSM

fp = '/content/data/padova-mestre-veneto.osm.pbf'
osm = OSM(fp)

Buildings

In [62]:
buildings = osm.get_buildings()

In [63]:
buildings.head()

,addr:city,addr:country,addr:housenumber,addr:housename,addr:postcode,addr:place,addr:street,email,name,opening_hours,...,source,start_date,wikipedia,id,timestamp,version,geometry,tags,osm_type,changeset
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,15807159,0,-1,"POLYGON ((12.23064 45.48267, 12.23072 45.48284...",None,way,NaN
1,None,None,None,None,None,None,None,None,None,None,...,None,None,None,19905963,0,-1,"POLYGON ((12.25963 45.50042, 12.25990 45.50090...","{""layer"":""1"",""leisure"":""sports_centre"",""sport""...",way,NaN
2,Venezia,None,11,None,30174,None,Via Sergio Gori,None,Teatro del Parco,None,...,None,None,None,19906045,0,-1,"POLYGON ((12.25901 45.49954, 12.25921 45.49959...","{""layer"":""2"",""name:de"":""Parktheater"",""wheelcha...",way,NaN
3,None,None,None,None,None,None,None,None,None,None,...,None,None,None,23243893,0,-1,"POLYGON ((12.33720 45.43232, 12.33721 45.43230...","{""bicycle"":""no"",""highway"":""pedestrian"",""lit"":""...",way,NaN
4,None,None,None,None,None,None,None,None,Tronchetto Parking,"PH,Mo-Su 00:00-24:00+",...,survey;Bing WMS,None,None,23243985,0,-1,"POLYGON ((12.30731 45.44375, 12.30738 45.44372...","{""access"":""yes"",""fee"":""yes"",""parking"":""multi-s...",way,NaN


In [64]:
#filter buildings
buildings = osm.get_buildings()
buildings = buildings[buildings['name'].notnull()]
buildings.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 2262 entries, 2 to 327401
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   addr:city           475 non-null    object  
 1   addr:country        32 non-null     object  
 2   addr:housenumber    562 non-null    object  
 3   addr:housename      24 non-null     object  
 4   addr:postcode       471 non-null    object  
 5   addr:place          38 non-null     object  
 6   addr:street         623 non-null    object  
 7   email               74 non-null     object  
 8   name                2262 non-null   object  
 9   opening_hours       101 non-null    object  
 10  operator            115 non-null    object  
 11  phone               154 non-null    object  
 12  ref                 1 non-null      object  
 13  url                 7 non-null      object  
 14  website             232 non-null    object  
 15  building            2262 non

In [66]:
# Construct ontology namespaces not known by RDFlib
OSMO = Namespace("http://www.dei.unipd.it/database2/customOSMOntology#")

In [67]:
# Create the graph
g = Graph()

# Bind the namespaces to a prefix
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("osmo", OSMO)

In [68]:
%%time
#measure execution time

for index, row in buildings.iterrows():
    # Create the node to add to the Graph
    # the node has 'building' + id as URI
    Building = URIRef(OSMO["building"+str(row['id'])])
    g.add((Building, RDF.type, OSMO.Node))
    g.add((Building, OSMO['hasId'], Literal(row['id'], datatype=XSD.integer)))
    g.add((Building, OSMO['BuildingType'], Literal(row['shop'], datatype=XSD.string)))
    g.add((Building, OSMO['BuildingType'], Literal(row['office'], datatype=XSD.string)))
    g.add((Building, OSMO['BuildingType'], Literal(row['amenity'], datatype=XSD.string)))
    g.add((Building, OSMO['opensAt'], Literal(row['opening_hours'], datatype=XSD.string)))
    g.add((Building, OSMO['name'], Literal(row['name'], datatype=XSD.string)))
    addr = '{}, {}, {}, {}'.format(row['addr:street'], row['addr:housenumber'], row['addr:city'], row['addr:postcode'])
    g.add((Building, OSMO['address'], Literal(addr, datatype=XSD.string)))
    g.add((Building, OSMO['website'], Literal(row['website'], datatype=XSD.string)))

CPU times: user 946 ms, sys: 11.4 ms, total: 958 ms
Wall time: 952 ms


In [69]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savepath + 'buildings.ttl', 'wb') as file:
    file.write(g.serialize(format='turtle', encoding='utf-8'))

--- saving serialization ---
CPU times: user 1.07 s, sys: 4.75 ms, total: 1.08 s
Wall time: 1.08 s
